In [ ]:
from StrategyUtils.strategy import get_predictions_dfs, StrategyTester, Strategy, save_strategy_df, load_strategy_df
from matplotlib import pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Determine wether to create_strategy_dfs or just load the previous ones

In [ ]:
just_load = True

In [ ]:
if just_load:
    pass
else:
    h4_train, h4_val, h4_test = get_predictions_dfs(240)
    h1_train, h1_val, h1_test = get_predictions_dfs(60)
    m15_train, m15_val, m15_test = get_predictions_dfs(15)

In [ ]:
if just_load is False:
    m15_train.head(1).index

In [ ]:
if just_load is False:
    m15_val.head(1).index

In [ ]:
if just_load is False:
    m15_test.head(1).index

In [ ]:
if just_load is False:
    m15_test.tail(1).index

# Train

In [ ]:
if just_load is True:
    train_df = load_strategy_df('nvae_train')
else:
    train_df = m15_train.copy() 

    train_df['h4_decision'] = h4_train.decision.shift(1)
    train_df['h4_decision'] = train_df.h4_decision.shift(-1)

    train_df['h1_decision'] = h1_train.decision.shift(1)
    train_df['h1_decision'] = train_df.h1_decision.shift(-1)

    train_df.fillna(method='ffill', inplace=True)
    train_df.rename(columns={'decision': 'm15_decision'}, inplace=True)
    
    save_strategy_df(train_df, 'nvae_train')

In [ ]:
# strategy 
train_df['short_signal'] = (train_df.h4_decision < 0) & (train_df.m15_decision > 0.65) & (train_df.m15_decision < 0.79)
train_df['close_long'] = (train_df.h4_decision > 0.3) & (train_df.m15_decision > 0.5)

train_df['close_short'] = (train_df.h4_decision < -0.3) & (train_df.m15_decision < -0.5)
train_df['long_signal'] = (train_df.h4_decision > 0) & ((train_df.m15_decision < -0.92) | ((train_df.m15_decision < -0.41) & (train_df.m15_decision > -0.82)))

train_df['Neutral'] = (train_df.short_signal == False) & (train_df.close_long == False) & (train_df.close_short == False) & (train_df.long_signal == False)

train_df['decision'] = train_df.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

train_decisions_df = train_df.loc[:, ['high', 'low', 'closing_price', 'decision']]
s1 = StrategyTester(train_decisions_df)

# buy and hold
train_df2 = train_df.copy()
train_df2['short_signal'] = False
train_df2['close_long'] = False

train_df2['close_short'] = False
train_df2['long_signal'] = True

train_df2['Neutral'] = False

train_df2['decision'] = train_df2.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

train_decisions_df2 = train_df2.loc[:, ['high', 'low', 'closing_price', 'decision']]

s2 = StrategyTester(train_decisions_df2)

## strategy 2

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_only.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')


# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_only.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy2: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')

## strategy 4

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_and_sell.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')

# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_and_sell.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy4: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')

# Validation

In [ ]:
if just_load is True:
    val_df = load_strategy_df('nvae_val')
else:
    val_df = m15_val.copy() 

    val_df['h4_decision'] = h4_val.decision.shift(1)
    val_df['h4_decision'] = val_df.h4_decision.shift(-1)

    val_df['h1_decision'] = h1_val.decision.shift(1)
    val_df['h1_decision'] = val_df.h1_decision.shift(-1)

    val_df.fillna(method='ffill', inplace=True)
    val_df.rename(columns={'decision': 'm15_decision'}, inplace=True)
    
    save_strategy_df(val_df, 'nvae_val')

In [ ]:
# strategy
val_df['short_signal'] = (val_df.h4_decision < 0) & (val_df.m15_decision > 0.65) & (val_df.m15_decision < 0.79)
val_df['close_long'] = (val_df.h4_decision > 0.3) & (val_df.m15_decision > 0.5)

val_df['close_short'] = (val_df.h4_decision < -0.3) & (val_df.m15_decision < -0.5)
val_df['long_signal'] = (val_df.h4_decision > 0) & ((val_df.m15_decision < -0.92) | ((val_df.m15_decision < -0.41) & (val_df.m15_decision > -0.82)))
val_df['Neutral'] = (val_df.short_signal == False) & (val_df.close_long == False) & (val_df.close_short == False) & (val_df.long_signal == False)

val_df['decision'] = val_df.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

val_decisions_df = val_df.loc[:, ['high', 'low', 'closing_price', 'decision']]

s1 = StrategyTester(val_decisions_df)

# buy and hold
val_df2 = val_df.copy()
val_df2['short_signal'] = False
val_df2['close_long'] = False

val_df2['close_short'] = False
val_df2['long_signal'] = True

val_df2['Neutral'] = False

val_df2['decision'] = val_df2.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

val_decisions_df2 = val_df2.loc[:, ['high', 'low', 'closing_price', 'decision']]

s2 = StrategyTester(val_decisions_df2)

## Strategy 2

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_only.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')

# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_only.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy2: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')

## Strategy 4

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_and_sell.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')

# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_and_sell.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy4: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')

# Test

In [ ]:
if just_load is True:
    test_df = load_strategy_df('nvae_test')
else:
    test_df = m15_test.copy() 

    test_df['h4_decision'] = h4_test.decision.shift(1)
    test_df['h4_decision'] = test_df.h4_decision.shift(-1)

    test_df['h1_decision'] = h1_test.decision.shift(1)
    test_df['h1_decision'] = test_df.h1_decision.shift(-1)

    test_df.fillna(method='ffill', inplace=True)
    test_df.rename(columns={'decision': 'm15_decision'}, inplace=True)
    
    save_strategy_df(test_df, 'nvae_test')

In [ ]:
# strategy 4
test_df['short_signal'] = (test_df.h4_decision < 0) & (test_df.m15_decision > 0.65) & (test_df.m15_decision < 0.79)
test_df['close_long'] = (test_df.h4_decision > 0.3) & (test_df.m15_decision > 0.5)

test_df['close_short'] = (test_df.h4_decision < -0.3) & (test_df.m15_decision < -0.5)
test_df['long_signal'] = (test_df.h4_decision > 0) & ((test_df.m15_decision < -0.92) | ((test_df.m15_decision < -0.41) & (test_df.m15_decision > -0.82)))
test_df['Neutral'] = (test_df.short_signal == False) & (test_df.close_long == False) & (test_df.close_short == False) & (test_df.long_signal == False)

test_df['decision'] = test_df.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

test_decisions_df = test_df.loc[:, ['high', 'low', 'closing_price', 'decision']]

s1 = StrategyTester(test_decisions_df)

# buy and hold
test_df2 = test_df.copy()
test_df2['short_signal'] = False
test_df2['close_long'] = False

test_df2['close_short'] = False
test_df2['long_signal'] = True

test_df2['Neutral'] = False

test_df2['decision'] = test_df2.apply(lambda x: np.argmax(x[6:11]), axis=1, raw=True)

test_decisions_df2 = test_df2.loc[:, ['high', 'low', 'closing_price', 'decision']]

s2 = StrategyTester(test_decisions_df2)

## Strategy 2

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_only.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')

# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_only.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy2: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')

## Strategy 4

In [ ]:
# plot strategy results
pnl_df = s1.generate_results(Strategy.four_states_buy_and_sell.value)
plt.figure(figsize=(40, 20))
plt.hlines(0, 0, len(pnl_df), color=['red'])
pnl_df['cpnl'] = pnl_df.pnl.cumsum()
pnl_df.cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df.cpnl.values, 'm-')

# buy and hold
pnl_df2 = s2.generate_results(Strategy.four_states_buy_and_sell.value)
pnl_df2['cpnl'] = pnl_df2.pnl.cumsum()
pnl_df2.live_cpnl.fillna(method='ffill', inplace=True)
plt.plot(pnl_df2.live_cpnl.values, 'b-')
plt.show()

print('strategy4: ')
print('profit: ', pnl_df.pnl.sum(), '$')
print('MDD: ', s1.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df[pnl_df.pnl > 0]) / len(pnl_df.dropna()), '%')
print(f'number of positions: {s1.n_positions}')

print('___________________________________')

print('BuyAndHold: ')
print('profit: ', pnl_df2.pnl.sum(), '$')
print('MDD: ', s2.mdd, '$')
print('percentage of profitable trades: ', 100 * len(pnl_df2[pnl_df2.pnl > 0]) / 1, '%')
print(f'number of positions: {s2.n_positions}')

plt.figure(figsize=(40, 20))
live_cpnl = pnl_df.btc_live_cpnl.fillna(method='ffill')
plt.plot(live_cpnl.values, 'm-')
plt.plot(pnl_df.close.values, 'b-')
plt.show()
print('1btc strategy2_profit: ', live_cpnl.values[-1], '$')
print('1btc BuyAndHold_profit: ', pnl_df.iloc[-1, pnl_df.columns.get_loc('close')], '$')